In [1]:
!pip install mediapipe opencv-python

  Using cached mediapipe-0.8.11-cp38-cp38-macosx_10_15_x86_64.whl (33.6 MB)


In [2]:
import cv2
import numpy as np
import mediapipe as mp

In [19]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_style = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [20]:
hands = mp_hands.Hands(static_image_mode = True,
                      max_num_hands = 2,
                      min_detection_confidence = 0.5)
files = ['./hand_figs/both_hand.jpeg']
image = cv2.flip(cv2.imread(files[0]), 1) # 헷갈릴 수 있으므로 좌우를 뒤집어야 함
image_bgr = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

annotated_image = image.copy()

results = hands.process(image_bgr)
print('Handness', results.multi_handedness)
print('Handness', results.multi_handedness[0].classification[0].score)

Handness [classification {
  index: 1
  score: 0.9574827551841736
  label: "Right"
}
, classification {
  index: 0
  score: 0.985072672367096
  label: "Left"
}
]
Handness 0.9574827551841736


In [21]:
# print('landmark', results.multi_hand_world_landmarks[0].landmark)
print('fingers :', results.multi_hand_world_landmarks[0].landmark[1])
print('fingers :', results.multi_hand_world_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP])

fingers : x: -0.028711356222629547
y: 0.06749913841485977
z: -0.004177887458354235

fingers : x: -0.047275591641664505
y: -0.07090231031179428
z: -0.04067772626876831



In [22]:
mp_drawing.draw_landmarks(image = annotated_image, 
                         landmark_list=results.multi_hand_landmarks[0], connections=mp_hands.HAND_CONNECTIONS, 
                         landmark_drawing_spec=mp_drawing_style.get_default_hand_landmarks_style(), 
                         connection_drawing_spec=mp_drawing_style.get_default_hand_connections_style())

mp_drawing.draw_landmarks(image = annotated_image, 
                         landmark_list=results.multi_hand_landmarks[1], connections=mp_hands.HAND_CONNECTIONS, 
                         landmark_drawing_spec=mp_drawing_style.get_default_hand_landmarks_style(), 
                         connection_drawing_spec=mp_drawing_style.get_default_hand_connections_style())

In [23]:
cv2.imshow('image', annotated_image)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

# webcam 을 이용한 손 detect

In [18]:
# For webcam input:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == 27:
            break
            
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1